기둥과 보 설치
---
https://school.programmers.co.kr/learn/courses/30/lessons/60061

- 기둥은 바닥 위에 있거나 보의 한쪽 끝 부분 위에 있거나, 또는 다른 기둥 위에 있어야 합니다.
- 보는 한쪽 끝 부분이 기둥 위에 있거나, 또는 양쪽 끝 부분이 다른 보와 동시에 연결되어 있어야 합니다.

build_frame의 원소는 [x, y, a, b]형태입니다.
- x, y는 기둥, 보를 설치 또는 삭제할 교차점의 좌표이며, [가로 좌표, 세로 좌표] 형태입니다.
- a는 설치 또는 삭제할 구조물의 종류를 나타내며, 0은 기둥, 1은 보를 나타냅니다.
- b는 구조물을 설치할 지, 혹은 삭제할 지를 나타내며 0은 삭제, 1은 설치를 나타냅니다.
- 벽면을 벗어나게 기둥, 보를 설치하는 경우는 없습니다.
- 바닥에 보를 설치 하는 경우는 없습니다.

In [161]:
class pillar:
    def __init__(self, x, y):
        self.type = 0
        self.x = x
        self.y = y
        self.up= None
        self.down= None
        self.left = None
        self.up_left = None
        self.center = None
    def return_connection(self):
        _ = []
        if self.up is not None:
            _.append(self.up)
        if self.down is not None:
            _.append(self.down)
        if self.left is not None:
            _.append(self.left)
        if self.up_left is not None:
            _.append(self.up_left)
        return _
        
class bo:
    def __init__(self, x, y):
        self.type = 1
        self.x = x
        self.y = y
        self.left= None
        self.right_down = None
        self.right = None
        self.down = None
        self.center = None
    def return_connection(self):
        _ = []
        if self.right_down is not None:
            _.append(self.right_down)
        if self.down is not None:
            _.append(self.down)
        if self.left is not None:
            _.append(self.left)
        if self.right is not None:
            _.append(self.right)
        return _
    
class map_:

    def __init__(self, n):
        self.bulidings = [[None] * (n+1) for i in range(n+1)]
        self.building_dict = {}
        self.id = 0
        self.n = n


    def check(self, x, y):
        building = self.building_dict[self.bulidings[y][x]]
        if building.type:
            if building.down is not None:
                if self.building_dict[building.down] is not None:
                    if self.building_dict[building.down].type == 0:
                        return True
            
            if building.right_down is not None:
                if self.building_dict[building.right_down] is not None:
                    if self.building_dict[building.right_down].type == 0:
                        return True


            if building.right is not None and self.building_dict[building.right] is not None:
                if self.building_dict[building.right].type == 1:
                    if building.left is not None and self.building_dict[building.left] is not None:  
                        if self.building_dict[building.left].type == 1:
                            return True

            return False  
         
        else:
            if y ==0:
                return True
            

            if building.down is not None and self.building_dict[building.down] is not None:
                if self.building_dict[building.down].type == 0:
                    return True
            


            if building.left is not None and self.building_dict[building.left] is not None:
                if self.building_dict[building.left].type == 1:
                    return True        

            return False



    def build(self, x, y , type_):
        if type_:
            self.bulidings[y][x] = self.id
            self.building_dict[self.id] = bo(x, y)
        else:
            self.bulidings[y][x] = self.id
            self.building_dict[self.id] = pillar(x, y)

        self.connect(x, y)
        # print(self.check(x,y), type_)
        if not self.check(x,y):
            self.bulidings[y][x] = None
            self.building_dict[self.id] = None
        
        else:
            self.id +=1

    def destroy(self, x, y, a):
        # for _ in self.bulidings[::-1]:
        #     print([f'{i: 2d}' if i is not None else '  ' for i in _])
        # print()

        type_ =  self.building_dict[self.bulidings[y][x]].type
        if type_ !=a:
            return 0
        x = self.building_dict[self.bulidings[y][x]].x
        y = self.building_dict[self.bulidings[y][x]].y
        buls = self.building_dict[self.bulidings[y][x]].return_connection()
        self.building_dict[self.bulidings[y][x]] = None
        self.bulidings[y][x] = None
        # print(buls)


        for bul in buls:
            connected_bul = self.building_dict[bul]
            if connected_bul is None:
                continue
            # print('기둥'if connected_bul.type==0 else '보', connected_bul.x, connected_bul.y)

            if not self.check(connected_bul.x,connected_bul.y):
                self.build(x, y , type_)
                break

    def connect(self, x, y):
        building = self.building_dict[self.bulidings[y][x]]

        # print(building.type, x, y)
        if building.type:
            if x>0:
                building.left= self.bulidings[y][x-1]

            building.right_down = self.bulidings[y-1][x+1]
            building.right = self.bulidings[y][x+1]
            building.down = self.bulidings[y-1][x]
            building.center = self.bulidings[y][x]
            
            # print('left : ', building.left.type if building.left is not None else '')
            # print('right_down : ', building.right_down.type if building.right_down is not None else '')
            # print('right : ', building.right.type if building.right is not None else '')
            # print('down : ', building.down.type if building.down is not None else '')


            if building.left is not None:
                if self.building_dict[building.left].type:
                     self.building_dict[building.left].right = self.id
            if building.right_down is not None:
                if self.building_dict[building.right_down].type ==0:
                    self.building_dict[building.right_down].up_left = self.id
                
            if building.right is not None:
                self.building_dict[building.right].left = self.id
                
            if building.down is not None:
                if self.building_dict[building.down].type == 0:
                    self.building_dict[building.down].up = self.id
                    
            if building.down is not None:
                if self.building_dict[building.down].type == 0:
                    self.building_dict[building.down].up = self.id
                
            

        else:

            building.up= self.bulidings[y+1][x]
            if y >0:
                building.down= self.bulidings[y-1][x]
            if x>0:
                building.left = self.bulidings[y][x-1]
                building.up_left = self.bulidings[y+1][x-1]
                
            # print('up : ', building.up.type if building.up is not None else '')
            # print('down : ', building.down.type if building.down is not None else '')
            # print('left : ', building.left.type if building.left is not None else '')
            # print('up_left : ', building.up_left.type if building.up_left is not None else '')

            if building.up is not None:
                self.building_dict[building.up].down = self.id
                    
            if building.down is not None:
                if self.building_dict[building.down].type == 0:
                    self.building_dict[building.down].up = self.id

            if building.left is not None:
                if self.building_dict[building.left].type:
                    self.building_dict[building.left].right = self.id
                
            if building.up_left is not None:
                if self.building_dict[building.up_left].type:
                    self.building_dict[building.up_left].right_down= self.id
                



def solution(n, build_frame):
    land = map_(n)

    for x, y, a, b in build_frame:
        # print([x, y],f"에 {'기둥'if a==0 else '보'} {'제거' if b==0 else '건설'}")
        if b:
            land.build(x, y , a)
        else:
            land.destroy(x, y, a)

    # for _ in land.bulidings[::-1]:
    #     print([f'{land.building_dict[i].type : 2d}' if i is not None else '  ' for i in _])
    # print()


    answer = [[structure.x, structure.y, structure.type ] for structure in land.building_dict.values() if structure is not None]
    return sorted(answer, key = lambda x: [x[0],x[1], -x[2]])

In [241]:
class pillar:
    def __init__(self, x, y):
        self.type = 0
        self.x = x
        self.y = y
        self.up= [None, None]
        self.down= [None, None]
        self.left = [None, None]
        self.up_left = [None, None]
        self.center = [None, None]
    def return_connection(self):
        _ = [i for i in self.up + self.down+ self.left + self.up_left + self.center if i is not None]
        return _
        
class bo:
    def __init__(self, x, y):
        self.type = 1
        self.x = x
        self.y = y
        self.left= [None, None]
        self.right_down = [None, None]
        self.right = [None, None]
        self.down = [None, None]
        self.center = [None, None]
    def return_connection(self):
        _ = [i for i in self.left + self.right_down+ self.right + self.down + self.center if i is not None]
        return _
    
class map_:

    def __init__(self, n):
        self.bulidings = [[[None, None] for j in range(n+1)]  for i in range(n+1)]
        self.building_dict = {}
        self.id = 0
        self.n = n


    def check(self, x, y,type_):
        building = self.building_dict[self.bulidings[y][x][type_]]
        if type_:
            if building.down[0] is not None:
                if self.building_dict[building.down[0]] is not None:
                    return True
            
            if building.right_down[0] is not None:
                if self.building_dict[building.right_down[0]] is not None:
                    return True


            if building.right[1] is not None and self.building_dict[building.right[1]] is not None:
                if building.left[1] is not None and self.building_dict[building.left[1]] is not None:  
                    return True

            return False  
         
        else:
            if y ==0:
                return True
            
            if building.down[0] is not None and self.building_dict[building.down[0]] is not None:
                return True
            
            if building.left[1] is not None and self.building_dict[building.left[1]] is not None:
                return True       
            if building.center[1] is not None and self.building_dict[building.center[1]] is not None:
                return True        

            return False



    def build(self, x, y , type_):
        self.bulidings[y][x][type_] = self.id

        if type_:
            self.building_dict[self.id] = bo(x, y)
        else:
            self.building_dict[self.id] = pillar(x, y)

        self.connect(x, y, type_)
        # print(self.check(x,y, type_))
        if not self.check(x,y,type_):
            self.bulidings[y][x][type_] = None
            self.building_dict[self.id] = None
        
        else:
            self.id +=1

    def destroy(self, x, y, a):
        # for _ in self.bulidings[::-1]:
        #     print([f'{i: 2d}' if i is not None else '  ' for i in _])
        # print()

        type_ =  self.building_dict[self.bulidings[y][x][a]].type
        if type_ !=a:
            return 0
        # x = self.building_dict[self.bulidings[y][x]].x
        # y = self.building_dict[self.bulidings[y][x]].y
        buls = self.building_dict[self.bulidings[y][x][a]].return_connection()
        self.building_dict[self.bulidings[y][x][a]] = None
        self.bulidings[y][x][a] = None
        # print(buls)


        for bul in buls:
            connected_bul = self.building_dict[bul]
            if connected_bul is None:
                continue
            # print('기둥'if connected_bul.type==0 else '보', connected_bul.x, connected_bul.y)

            if not self.check(connected_bul.x,connected_bul.y, connected_bul.type):
                self.build(x, y , type_)
                break

    def connect(self, x, y, type_):
        building = self.building_dict[self.bulidings[y][x][type_]]

        # print(building.type, x, y)
        if type_:
            if x>0:
                building.left= self.bulidings[y][x-1]

            building.right_down = self.bulidings[y-1][x+1]
            building.right = self.bulidings[y][x+1]
            building.down = self.bulidings[y-1][x]
            building.center = self.bulidings[y][x]
            
            # print('left : ', building.left.type if building.left is not None else '')
            # print('right_down : ', building.right_down.type if building.right_down is not None else '')
            # print('right : ', building.right.type if building.right is not None else '')
            # print('down : ', building.down.type if building.down is not None else '')


            if building.left[1] is not None:
                if self.building_dict[building.left[1]] is not None:
                     self.building_dict[building.left[1]].right = self.bulidings[y][x]
            if building.right_down[0] is not None:
                if self.building_dict[building.right_down[0]] is not None:
                    self.building_dict[building.right_down[0]].up_left = self.bulidings[y][x]
                
            if building.right[0] is not None:
                self.building_dict[building.right[0]].left = self.bulidings[y][x]
            if building.right[1] is not None:
                self.building_dict[building.right[1]].left = self.bulidings[y][x]

                
            if building.down[0] is not None:
                if self.building_dict[building.down[0]] is not None:
                    self.building_dict[building.down[0]].up = self.bulidings[y][x]
                
            if building.center[0] is not None:
                if self.building_dict[building.center[0]] is not None:
                    self.building_dict[building.center[0]].center = self.bulidings[y][x]
        else:

            building.up= self.bulidings[y+1][x]
            if y >0:
                building.down= self.bulidings[y-1][x]
            if x>0:
                building.left = self.bulidings[y][x-1]
                building.up_left = self.bulidings[y+1][x-1]
            building.center = self.bulidings[y][x]
                
            # print('up : ', building.up.type if building.up is not None else '')
            # print('down : ', building.down.type if building.down is not None else '')
            # print('left : ', building.left.type if building.left is not None else '')
            # print('up_left : ', building.up_left.type if building.up_left is not None else '')

            if building.up[0] is not None:
                self.building_dict[building.up[0]].down = self.bulidings[y][x]
            if building.up[1] is not None:
                self.building_dict[building.up[1]].down = self.bulidings[y][x]
                    
            if building.down[0] is not None:
                if self.building_dict[building.down[0]] is not None:
                    self.building_dict[building.down[0]].up = self.bulidings[y][x]

            if building.left[1] is not None:
                if self.building_dict[building.left[1]] is not None:
                    self.building_dict[building.left[1]].right = self.bulidings[y][x]
                
            if building.up_left[1] is not None:
                if self.building_dict[building.up_left[1]] is not None:
                    self.building_dict[building.up_left[1]].right_down = self.bulidings[y][x]

            if building.center[1] is not None:
                if self.building_dict[building.center[1]] is not None:
                    self.building_dict[building.center[1]].center = self.bulidings[y][x]


def solution(n, build_frame):
    land = map_(n)

    for x, y, a, b in build_frame:
        # print([x, y],f"에 {'기둥'if a==0 else '보'} {'제거' if b==0 else '건설'}")
        if b:
            land.build(x, y , a)
        else:
            land.destroy(x, y, a)

        # for _ in land.bulidings[::-1]:
        #     print([[str(land.building_dict[i[0]].type) if i[0] is not None else ' ',
        #             str(land.building_dict[i[1]].type) if i[1] is not None else ' '] for i in _])
        # print()


    answer = [[structure.x, structure.y, structure.type ] for structure in land.building_dict.values() if structure is not None]
    return sorted(answer, key = lambda x: [x[0],x[1],x[2]])

In [242]:
solution(5, [[1, 0, 0, 1], [1, 1, 0, 1], [1, 1, 1, 1], [0, 1, 1, 1],
             [2, 1, 0, 1], [0, 1, 0, 1], [2, 2, 0, 1], [1,3,1,1]])

[[0, 1, 0],
 [0, 1, 1],
 [1, 0, 0],
 [1, 1, 0],
 [1, 1, 1],
 [1, 3, 1],
 [2, 1, 0],
 [2, 2, 0]]

In [238]:
solution(5, 	[[1,0,0,1],[1,1,1,1],[2,1,0,1],[2,2,1,1],[5,0,0,1],[5,1,0,1],[4,2,1,1],[3,2,1,1]])

[1, 0] 에 기둥 건설
[[' ', ' '], [' ', ' '], [' ', ' '], [' ', ' '], [' ', ' '], [' ', ' ']]
[[' ', ' '], [' ', ' '], [' ', ' '], [' ', ' '], [' ', ' '], [' ', ' ']]
[[' ', ' '], [' ', ' '], [' ', ' '], [' ', ' '], [' ', ' '], [' ', ' ']]
[[' ', ' '], [' ', ' '], [' ', ' '], [' ', ' '], [' ', ' '], [' ', ' ']]
[[' ', ' '], [' ', ' '], [' ', ' '], [' ', ' '], [' ', ' '], [' ', ' ']]
[[' ', ' '], ['0', ' '], [' ', ' '], [' ', ' '], [' ', ' '], [' ', ' ']]

[1, 1] 에 보 건설
[[' ', ' '], [' ', ' '], [' ', ' '], [' ', ' '], [' ', ' '], [' ', ' ']]
[[' ', ' '], [' ', ' '], [' ', ' '], [' ', ' '], [' ', ' '], [' ', ' ']]
[[' ', ' '], [' ', ' '], [' ', ' '], [' ', ' '], [' ', ' '], [' ', ' ']]
[[' ', ' '], [' ', ' '], [' ', ' '], [' ', ' '], [' ', ' '], [' ', ' ']]
[[' ', ' '], [' ', '1'], [' ', ' '], [' ', ' '], [' ', ' '], [' ', ' ']]
[[' ', ' '], ['0', ' '], [' ', ' '], [' ', ' '], [' ', ' '], [' ', ' ']]

[2, 1] 에 기둥 건설
[[' ', ' '], [' ', ' '], [' ', ' '], [' ', ' '], [' ', ' '], [' ', ' ']]
[[' '

[[1, 0, 0],
 [1, 1, 1],
 [2, 1, 0],
 [2, 2, 1],
 [3, 2, 1],
 [4, 2, 1],
 [5, 0, 0],
 [5, 1, 0]]